# Research markdown generator for academicpages

Takes a TSV of research with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](https://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `research.py`. Run either from the `markdown_generator` folder after replacing `research.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: proj-date, title, excerpt, source, url_slug, project_url,  code_url, image with a header at the top. 
- `excerpt` and `image` can be blank, but the others must have values. 
- `proj-date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/research/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!cat projects.tsv

proj_date	title	excerpt	source	url_slug	code_url	media	thumbnail	paper_url	   	
2020-12-20	Snoring Detection on a Microcontroller	Snoring is related to a common medical condition that can lead to many serious health issues including diabetes, stroke, and depression. Due to the negative health impacts of such medical condition, it is crucial for people to know whether they snore and understand their snoring patterns and triggers of snoring. In this paper, we propose a bed-side snoring detection program on microcontroller device that automatically identifies snoring sounds.	Rotaru, A., Kelly Zhang, Jiayu Yao (2020). "Snoring Detection on a Microcontroller" <i>Harvard CS249r: Tiny Machine Learning (TinyML) </i>.	snoring-detection	https://github.com/adrianagaler/Snoring-Detection	https://www.youtube.com/watch?v=aQpIooBEGsA	../images/snoring.png	https://wwww.adrianarotaru.github.io/files/snoring.pdf
2020-05-12	Unity Bloxorz Game (Open GL Graphics)	A game akin to Bloxorz in the Unity game en

## Import pandas

We are using the very handy pandas library for dataframes.

In [2]:
# ! pip install -U pandas
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [3]:
research = pd.read_csv("projects.tsv", sep="\t", header=0)
research


,proj_date,title,excerpt,source,url_slug,code_url,media,thumbnail,paper_url,,Unnamed: 10
0,2020-12-20,Snoring Detection on a Microcontroller,Snoring is related to a common medical conditi...,"Rotaru, A., Kelly Zhang, Jiayu Yao (2020). ""Sn...",snoring-detection,https://github.com/adrianagaler/Snoring-Detection,https://www.youtube.com/watch?v=aQpIooBEGsA,../images/snoring.png,https://wwww.adrianarotaru.github.io/files/sno...,NaN,NaN
1,2020-05-12,Unity Bloxorz Game (Open GL Graphics),A game akin to Bloxorz in the Unity game engin...,"Rotaru, A., Revel Sebastian (2021). ""Unity Blo...",game-unity,https://github.com/adrianagaler/CS175-FinalPro...,https://github.com/adrianagaler/CS175-FinalPro...,../images/game.png,https://wwww.adrianarotaru.github.io/files/gam...,NaN,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [5]:
import os
import numpy as np
for row, item in research.iterrows():
    md_filename = str(item.proj_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.proj_date) + "-" + item.url_slug
    year = item.proj_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: projects"""
    
    md += """\npermalink: /project/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.proj_date)     
    md += "\ncodeurl: '" + html_escape(item.code_url) + "'"
    
    md += "\nsource: '" + html_escape(item.source) + "'"

    md += "\n---"
    
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.code_url)) > 5:
        md += "\n[Access code here](" + item.code_url + ")\n" 

    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n"

    if not pd.isnull(item.thumbnail) and not pd.isnull(item.media):
        md += f"\n[![{item.title}]({item.thumbnail})]({item.media})\n"

    md_filename = os.path.basename(md_filename)
       
    with open("../_projects/" + md_filename, 'w') as f:
        f.write(md)

These files are in the research directory, one directory below where we're working from.

In [6]:
!ls ../_projects/

2020-05-12-game-unity.md        2020-12-20-snoring-detection.md
